# Load and initialize


settings


In [ ]:
# sample number

# j = '_arcsinh'+'18.2'+'a'# '18.2'+'a'#
# j = '18.2'+'a'#  '_arcsinh'+'18.2'+'a'#'14.2'+'a'#'_raw'+'14.2'+'a'#
            
# '7.1a','13a','14a','15a','17a','18a','19a','20a','8.1a'

j = '14.2'+'a'
# j = '7.1' +'a'

group_ind = 2
dir_data1 = 'Data'

args = {
        # create visualization (umap, dbscan are always shown)
        'visualize':True,'plotUMAP' : True,
        # show figures in notebook
        'show' : True, 'saveSVG' :False,
        # calculate umap
        'calc_umap' : False
        }
create_adjusted = False
compare = False# compare 11,stroma and rest of the cells
print(j, 'ind:',group_ind, dir_data1, args, create_adjusted)

envirement


In [ ]:

from tqdm import tqdm_notebook, tqdm
import datetime

import pandas as pd
from matplotlib.pyplot import rc_context
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image, display
import sys
import pathlib
import os
from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)
import time
start = time.process_time()
date = datetime.date.today().strftime("%d%m%Y")
# import time

# import matplotlib

plt.rcParams["figure.figsize"] = (5.0, 4.0)  # Set default size of plots.
plt.rcParams["image.interpolation"] = "nearest"
plt.rcParams["image.cmap"] = "gray"
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 180)  # according to screen width
# from pandas.core.base import PandasObject
# PandasObject.view = view #allows view meth pd.view

# from scipy import signal, stats
# import seaborn as sns

# from sklearn import metrics

# from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
# from sklearn.metrics import silhouette_samples,silhouette_score
# from sklearn.neighbors import kneighbors_graph

# from lmfit import minimize, Parameters


parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

sys.path.append(parent_dir+'/functions/')

# from epinuc import *
from usefull_functions import *
from functions import *
from plot_functions import *
from clustering_functions import *
from xg_shap import *    


%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%autoreload 2


# TF_ENABLE_ONEDNN_OPTS=0


load data; normalized (CD45-)


In [ ]:
j, group_ind, dir_data1,args = getJ(j, group_ind, dir_data1,args)



dir_data = os.path.abspath(os.path.join(    parent_dir, os.pardir))+f'/{dir_data1}/'
dir_plots = os.path.abspath(os.path.join( parent_dir, os.pardir))+f'/_sample_{j}_{dir_data1}_{date}/'
dir_index =  '/home/yishai/breast_cancer/indexes/'

folderExists(dir_plots)
settings = (dir_plots, args['show'], args['saveSVG'])

if 'b' not in j:
    singleSample = True
else:
    singleSample = False
print(f'singleSample is: {singleSample}')

# k_orig = pickle_load('k'+j+'_dict',dir_data )['k']
# k = k_orig.copy()
# CAll=pd.concat([k]).copy()

k = pickle_load('k'+j+'_dict', dir_data)['k'].reset_index(drop=True)
CAll = pd.concat([k.copy()])
print(f'k size: {len(k)}')


names_dict = pickle_load('k'+j+'_names_dict', dir_data)
if '.2a' in j: #remove them from the names_dict; they need to be only in cellcycle
    names_dict = removeFeatures(names_dict.copy(),remove_features = ['CyclinB1', 'pRB','H3S28p','DNA2'])#


NamesAll = names_dict['NamesAll']
CellIden = names_dict['CellIden']
EpiCols = names_dict['EpiCols']
Core = names_dict['Core']
CellIden_EpiCols = CellIden + EpiCols

# dict = None

groups = [['CellIden', 'EpiCols', 'CellIden+EpiCols',],
          [CellIden, EpiCols, CellIden_EpiCols]]

names = [[f'samp{j}_CellIden_', f'samp{j} CellIden Based'],
         [f'samp{j}_EpiCols_', f'samp{j} EpiCols Based'],
         [f'samp{j}_CellIden+EpiCols_', f'samp{j} CellIden+EpiCols Based'],
         ]

params = ['iden', 'epi', 'all',]

if '.2a'in j:
    groups[0].append( 'CellCycle')   
    groups[1].append( ['CyclinB1', 'pRB','H3S28p','DNA2'] )   
    names.append(   [f'samp{j}_CellCycle_', f'samp{j} CellCycle Based'])
    params.append('cell')
    print('cellcycle added')

Visualize data

no point to print data.describe() as it is whitened (std~1, mean~0)


In [ ]:
# if show:

#     for features in [NamesAll,Core,CellIden,EpiCols,CellIden_EpiCols]:
#         print(len(features))
#         print(features)


# Cell Identity


In [ ]:

figname, title = names[group_ind]
name, featureGroup = groups[0][group_ind], groups[1][group_ind]
print(name, title, figname)


### UMAP

plot for each feature according to its intensity


In [ ]:

min_dist, n_neighbors = getValsCsv(    dir_data, vars=[params[group_ind], 'umap', j])
# min_dist, n_neighbors = 0.02,10
print(f'min_dist:{min_dist}, n_neighbors:{n_neighbors}')
# calc_umap = True
if args['calc_umap']:
    umapData = calculate_umap(CAll[featureGroup].copy(), n_neighbors, min_dist)
    pickle_dump(f'umapData_{figname}', umapData, dir_data)
else:
    umapData = pickle_load(f'umapData_{figname}', dir_data)
drawUMAP(umapData, ['H4'], CAll, settings,
         title=f'min_dist = {min_dist}, n_neighbors = {n_neighbors}',
         Figname='1_'+figname+'UMAP_(core)')



In [ ]:
# len(umapData)

### Dbscan and clusters


Dbscan


In [ ]:
# labels__ = pickle_load(f'dbLabels_samp14.2a_CellIden+EpiCols_', dir_data) 


# ind = list(k.index)
# ind=np.random.choice(ind, replace = False, size = int(len(ind)/10)) #dilute sub sample so view is sparse
# dbscanData, labels, core_samples_mask  = calculate_dbscan(umapData[ind],ind, eps, min_samples)
eps, min_samples = getValsCsv(dir_data, vars=[params[group_ind], 'db', j])
# eps,min_samples =0.20, 55
print(f'{eps}, {min_samples}')

dbscanData, labels, core_samples_mask  = calculate_dbscan(umapData,list(k.index), eps, min_samples); k['Clust'] = labels

if args['calc_umap']:
    umapData = calculate_umap(CAll[featureGroup].copy(), n_neighbors, min_dist)
    pickle_dump(f'umapData_{figname}', umapData, dir_data)
else:
    umapData = pickle_load(f'umapData_{figname}', dir_data)

colors = drawDbscan(dbscanData, labels, core_samples_mask, settings,
                    title=f'min_samples = {min_samples}, eps = {eps}',
                    figname='1_'+figname+'dbscan')
pickle_dump(f'dbLabels_{figname}', labels, dir_data);pickle_dump(f'dbColors_{figname}', colors, dir_data)

# clustering cellcycle 

bimodal gassian fit

In [ ]:
if '.2a' in j and group_ind == 3:
    labels = None# verify that the labels from previous are not used
    if '_arcsinh' in j:
        from gaussian_fit import *


        x, y = sns.kdeplot(k['pRB'].copy(),label='label').lines[0].get_data()
        y =  np.asarray(y)/.2 if '18.2' in j else  np.asarray(y)**.5 #highlight the gaussians diffrence
        # plt.plot(x,y)

        cutoff = bimodal_minima(x,y,maxfev = 5000,plot=True)


        labels = np.zeros(len(k)).astype(int)
        labels[k['pRB']>cutoff] = 1

        pickle_dump(f'Labels_{figname}', labels, dir_data)
        
    try:labels = pickle_load(f'Labels_samp_arcsinh{j}_CellCycle_', dir_data) 
    except:pass
    names = ['pRB_low','pRB_high']
    colors = draw_clusters(data = umapData.copy(),labels = labels,title='cutoff clusters',figname='cutoff_clusters',settings = settings,names =names)   


    k['Clust'] = [names[l] for l in labels]#labels

kmeans

In [ ]:
# from clustering_functions import *

# if '.2a' in j and group_ind == 3:
#     if '_arcsinh' in j:
#         # high prb
#         labels = kmeans_fit(np.hstack ((umapData.copy(),CAll[[ 'pRB']].copy()))
#                             ,n_clusters=2,n_init=10,max_iter=300)
#         colors = draw_clusters(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters',settings = settings,
#                 names =['pRB- ','pRB+']  )
#         ind,_ = highClust_ind(k['pRB'],labels)


#         # high CyclinB1
#         labels1 = kmeans_fit(CAll[['CyclinB1']].loc[ind].copy(),n_clusters=2,n_init=10,max_iter=300)
#         ind11,ind2 = highClust_ind(k.loc[ind]['CyclinB1'],labels1)
#         labels[ind11] = 2#labels1[ind2]+2

#         # high H3S28p
#         labels2 = kmeans_fit(CAll[['H3S28p']].loc[ind].copy(),n_clusters=2,n_init=10,max_iter=300)
#         ind12,ind2 = highClust_ind(k.loc[ind]['H3S28p'],labels2)
#         labels[ind12] = 3#labels2[ind2]+4

#         ind13= [i for i in ind12 if i not in ind11]
#         labels[ind13] = 4
        
#     # load arcsin data
      
#         colors = draw_clusters(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters',settings = settings,
#                 names =['pRB+ not M&G2','pRB- (GO)','pRB+ G2','pRB+ M','pRB+ G2&M']  )
    
    
#         labels[ind12] = 2#labels2[ind2]+2
#         pickle_dump(f'kmeanLabels_{figname}', labels, dir_data)
#         # pickle_dump(f'kmeanColors_{figname}', colors, dir_data)
#         labels,labels2 = None,None
#     names =['not M&G2','pRB- (GO)','M&G2']
#     labels = pickle_load(f'kmeanLabels_samp_arcsinh14.2a_CellCycle_', dir_data) 
#     # colors = pickle_load(f'kmeanColors_samp_arcsinh14.2a_CellCycle_', dir_data)
#     colors = draw_clusters(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters1',settings = settings,
#                 names =names)   
    
#     k['Clust'] = [names[l] for l in labels]#labels



In [ ]:
# from plot_functions import *
 
# # colors = draw_kmeans(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters1',settings = settings,
# #             names =names,crossLabels = [labels__,1,None])

# for i in range (3):
#     print(i)
#     draw_kmeans(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters1',settings = settings,
#             names =names,crossLabels = [labels__,1,i])



adjusted dataset


In [ ]:

from plot_functions import *
    

# create_adjusted = True
if group_ind==2 and create_adjusted: #(  or ( '.2' in j))
    if ('b' in j and 'a' not in j):
        # remove -1 cluster
        indArr,idx_adj,idx_stroma = createAdjustedDataset(k,j,dbscanData, labels,core_samples_mask,settings,figname,
                            clustersToRemove=[],limits = (-1.8,None,None,None),drawFunc = drawDbscan,contained_samples = False)
        k = k.copy().reset_index(drop = True).loc[idx_adj];dbscanData, labels,core_samples_mask = dbscanData[idx_adj], labels[idx_adj],core_samples_mask[idx_adj]
        
        indArr,idx_adj,idx_stroma = createAdjustedDataset(k,j,dbscanData, labels,core_samples_mask,settings,figname,
                            clustersToRemove=[],limits = (None,2,None,None),drawFunc = drawDbscan,contained_samples = True)
    elif ('.2' in j):
        # remove -1 cluster
        indArr,idx_adj,idx_stroma = createAdjustedDataset(k,j,dbscanData, labels,core_samples_mask,settings,figname,
                            clustersToRemove=[],limits = (-1,None,None,None),drawFunc = drawDbscan,contained_samples = False)
        

    for kInd,ind in indArr.items():
        print(f'samp:{kInd}, len:{len(ind)}')     
        pickle_dump(f"{kInd}_indexes", ind,dir_index)
                            
    for i,idx in zip(['a','s'],[idx_adj,idx_stroma]):
        kInd = f'{j}{i}'
        newK = k.copy().reset_index(drop = True).loc[idx]
        dict = None;dict ={}
        dict['k'] = newK.copy()
        pickle_dump(f'k{kInd}_dict', dict,dir_data)
        pickle_dump(f'k{kInd}_names_dict', names_dict,dir_data)
    # print(len(k))
    # print(len(newK))
    # k = newK.copy()
    # CAll = pd.concat([k.copy()])


      
    indArr = None
    # K = pickle_load(f'k{sampNum_}_dict', dir_data)['k'].reset_index(drop=True)
    # ind_samp = [i for i, j in zip(k['ind'],k['samp'] == sampNum) if j]
    # ind_samp = [ K.index[K['ind']==i][0] for i in ind_samp ]
    # 7933


clusters

### HeatMap by clusters


In [ ]:

if args['visualize']:
    
    for group_name, group in zip(groups[0], groups[1]):       
        HeatMap(k[labels != -1], group, settings,clustFeature='Clust',
                title='HeatMap ' + title + ': '+group_name,
                figname='1_'+figname+'ClustHeatMap_by_'+group_name)
    


specific clusters and features

In [ ]:
# from plot_functions import *
# if args['visualize']:
#     clusters =[0,1]
#     group_name, group = 'specific_markers', ['H3K9me2', 'H3K4me3','H3K36me3','H4K20me3']
#     kk=k.copy()
#     # -----------------
#     k = k.loc[[True if c in clusters else False for c in k['Clust'] ]]
#     HeatMap(k, group, settings,clustFeature='Clust',
#     title='HeatMap ' + title + ': '+group_name,
#                 figname='1_'+figname+'ClustHeatMap_by_'+group_name)
#     # ---------------------
#     k=kk.copy()
#     kk = None



### HeatMap by sample


In [ ]:
from plot_functions import *
if args['visualize'] and not singleSample:
    for group_name, group in zip(groups[0], groups[1]):
    # group_name, group = groups[0][-1], groups[1][-1]
        HeatMap(k[labels != -1], group, settings, clustFeature='samp',
                title='HeatMap ' + title + ': ' + group_name+' by_sample',
                figname='1_'+figname+'SampHeatMap_' + group_name)


In [ ]:
if args['visualize'] and j =='b2345' and compare:# compare sample 11,stroma and rest of the cells
    # load from file

    df_a = pickle_load('k'+'b2345a'+'_dict', dir_data)['k'].reset_index(drop=True)
    ind = df_a['samp']==11
    df11= df_a[ind].copy().reset_index(drop=True);df11['samp'] = '11'
    df_a= df_a[~ind].copy().reset_index(drop=True);df_a['samp'] = 'all'
    df_s = pickle_load('k'+'b2345s'+'_dict', dir_data)['k'].reset_index(drop=True);df_s['samp'] = 'stroma'

    df = pd.DataFrame(columns = df11.columns)
    for df_ in [df11,df_a,df_s]:
        print(len(df_),df_['samp'].unique())
        df = pd.concat([df,df_.copy()], ignore_index=True,axis=0,)
    df=df.reset_index(drop=True)

    from plot_functions import *
    for group_name, group in zip(groups[0], groups[1]):#df[labels != -1]
    # group_name, group = groups[0][-1], groups[1][-1]
        HeatMap(df, group, settings, clustFeature='samp',
                title='HeatMap ' + title + ': ' + group_name+' stroma_vs_11_vsrest',
                figname='1_'+figname+'stroma_vs_11_vsrest_' + group_name)


In [ ]:
# if compare:# compare sample 11,stroma and rest of the cells
#     # load from file

#     df_a = pickle_load('k'+'b2345a'+'_dict', dir_data)['k'].reset_index(drop=True)
#     df_s = pickle_load('k'+'b2345s'+'_dict', dir_data)['k'].reset_index(drop=True)
#     df11= df_a[df_a['samp']==11].copy().reset_index(drop=True)
#     # df = pd.DataFrame(columns = k1['11'].columns)
#     # for i,kk in k1.items():
#     #     kk['samp'] = i
#     #     df = pd.concat([df,kk.copy()], ignore_index=True,axis=0,)


    







#     keys = [f'{int(i)}a'  for i in k['samp'].unique() if '.1' not in str(i)];keys.remove('11a')
#     keys = keys[:2]

#     kk={}

#     for i in keys:
#         i_adj = f'{i}a'

#         K = k[i].copy()
#         # print(i_adj)
#         # k[i_adj]= subsample_k(k[i].copy(),i_adj,dir_indexes)
#         names[i_adj] = names[i].copy()
#         idx = pickle_load(f"{i_adj}_indexes",dir_index)
        
#         newIdx = [ K.index[K['ind']==i][0] for i in idx ]
#         kk[i_adj]=K.loc[newIdx].copy()
#         print (f'original size: {len(k[i])}, new size: {len(k[i_adj])} indexes loaded from file')
#         # del k[i];del names[i]






#     print(keys)
#     k1,names1= createAppendDataset(kk.copy(),names_dict.copy(),keys,[] )

# #     for group_name, group in zip(groups[0], groups[1]):
# #     # group_name, group = groups[0][-1], groups[1][-1]
# #         HeatMap(k[labels != -1], group, settings, clustFeature='samp',
# #                 title='HeatMap ' + title + ': ' + group_name+' by_sample',
# #                 figname='1_'+figname+'SampHeatMap_'+group_name)




### MeanDist


In [ ]:

if args['visualize']:
    for group_name, group in zip(groups[0], groups[1]):
        MeanDist(k[labels == 0], k[labels == 1], group, settings,
                title='MeanDist ' + title + ': '+group_name,
                figname='1_'+figname+'ClustMeanDist_by_'+group_name)


## sample percentage


by sample

In [ ]:
# from plot_functions import *
if args['visualize'] and not singleSample:
    # unique by_sample values
    for sampNum in np.unique(k['samp']):
        ind_samp = [i for i, j in enumerate(k['samp'] == sampNum) if j]
        labels_samp = np.ones(len(ind_samp))*sampNum
        drawUMAPbySample(umapData, k, ind_samp, labels_samp, settings,
                         title=title+f' UMAP - by_sample={sampNum}',
                         Figname='4_'+figname+f'UMAP_sample{sampNum}_by_sample')
    arr = []
    clustNums = np.unique(k.Clust)
    clustNums = clustNums[clustNums != -1]
    for clustNum in clustNums:
        ind = k.Clust == clustNum
        PercentageBySample = ClustPercentageBySample(k[ind], 'samp')

        arr.append([clustNum, PercentageBySample])
    saveCsv(dir_plots, figname, arr)


all samples

In [ ]:
from plot_functions import *


if args['visualize'] and not singleSample:
    # remove 4.1,7.1 and subsample 
    uniq = [ i   for i in k['samp'].unique() if i not in [4.1,7.1,8.1]]
    ind_samp =[]
    for i in uniq:
        ind  = k.index.values[k['samp']==i]
        ind = list(np.random.choice(ind, replace = False, size = int(20000/len(uniq))))
        ind_samp += ind
    ind_samp = np.asarray(ind_samp)

    # ind_samp =k.index.values[[ True  if i in uniq else False for i in k['samp']]]
    # ind_samp=np.random.choice(ind_samp, replace = False, size = int(len(ind_samp)/len(uniq)))
    labels_samp = np.asarray(k['samp']).astype(int)[ind_samp]

    # ind_samp = k.index.values
    # labels_samp = np.asarray(k.by_sample)[ind_samp]


    colors_samp = np.zeros((np.max(labels_samp)+2,4))
    # hex colors
    hex =  ['a6cee3','1f78b4','b2df8a','33a02c','fb9a99','e31a1c','fdbf6f','ff7f00','cab2d6','6a3d9a','ffff99','b15928']
    # hex = ['00B3A4','3185FC','DB1374','490092','FEB6DB','F98510','E6C220','BFA180','920000','461A0A', 
    #        'FF0000','FF00FF',]
        #    '00FFFF','0000FF','000080','008000','00FF00',   '808000','800000','800080','808080','C0C0C0','FFFFFF','000000']    
    colors_samp[1+np.unique(labels_samp)] = np.asarray([hex_to_rgba(h) for h in hex])[:np.unique(labels_samp).shape[0]]
    drawUMAPbySample(umapData, k, ind_samp, labels_samp, settings,colors_samp,backgroundColor = 'white',
                        title=title+f' UMAP - by_sample=ALL',
                        Figname='4_'+figname+f'UMAP_sampleAll_by_sample')
 


by cluster

In [ ]:
from plot_functions import *
if args['visualize'] and not singleSample and 'a' in j:#plot on adjusted
    # plt.scatter(umapData[:,0],umapData[:,1],c = 'lightgrey', alpha=0.2,s=2)
    uniq = [int(i) if (np.round(i,1) - np.round(i,0)==0) else float(i)  for i in np.unique(k['samp'])]
    # print(uniq)
    # unique by_sample values
    for sampNum in uniq:
        # sampNum = uniq[-1]
        sampNum_ =f'{sampNum}a' if 'a' in j else sampNum     #load adjusted

        

        # ind_umap = [i for i, j in enumerate(k.by_sample == sampNum) if j]
        # labels_samp = np.asarray(pickle_load(f'dbLabels_samp{sampNum}_{name}_', dir_data))
        
        # sample indexs which are the full dataset - filter db labels by it
        K = pickle_load(f'k{sampNum_}_dict', dir_data)['k'].reset_index(drop=True)
        ind_samp = [i for i, j in zip(k['ind'],k['samp'] == sampNum) if j]
        ind_samp = [ K.index[K['ind']==i][0] for i in ind_samp ]

        K = None
        # corresponding indexes in the full data set - where each point in the full umap
        ind_umap = [i for i, j in zip(k.index,k['samp'] == sampNum) if j]
        labels_samp = np.asarray(pickle_load(f'dbLabels_samp{sampNum_}_{name}_', dir_data))[ind_samp]
        # colors_samp = pickle_load(f'dbColors_samp{sampNum}_{name}_', dir_data)
        colors_samp = np.vstack(([0,0,0,1],np.asarray(pickle_load(f'dbColors_samp{sampNum_}_{name}_', dir_data))))
        # pickle_dump(f'dbColors_{figname}', colors, dir_data)

        drawUMAPbySample(umapData, k, ind_umap, labels_samp, settings,colors_samp,
                         title=title+f' UMAP - by_sample={sampNum} clusters',
                         Figname='4_'+figname+f'UMAP_sample{sampNum}_by_sample_clusters')
        
# [4, 4.1, 5, 7, 7.1, 8, 11, 13, 14, 15]

7933

.2_adj

In [ ]:
# from plot_functions import *
if '.2a' in j and group_ind==2:  
    # k['kmean_labels'] = pickle_load(f'kmeanLabels_samp{j}_CellCycle_', dir_data).astype(float)
    k['cellcycle_labels'] = pickle_load(f'Labels_samp_arcsinh{j}_CellCycle_', dir_data).astype(float)
    names =['pRB-','pRB+']

    # colors = draw_kmeans(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters',settings = settings,
    #             names =['not M&G1','pRB- (GO)','M&G1'])

    arr = []
    clustNums = np.unique(k.Clust)
    clustNums = clustNums[clustNums != -1]
    for clustNum in clustNums:
        ind = k.Clust == clustNum #filter by db clusters
        PercentageBySample = ClustPercentageBySample(k[ind], 'cellcycle_labels',names =names)#names =['not M&G2','pRB- (GO)','M&G2']#names = ['pRB+ not M&G2','pRB- (GO)','2','pRB+ G2','4','pRB+ M','pRB+ G2&M']
        arr.append([clustNum, PercentageBySample])
            


    saveCsv(dir_plots, f'{j}_cellcycle_labels', arr)
 



In [ ]:
    # for sampNum in clustNums:
    #     # sampNum = uniq[-1]

    #     # ind_umap = [i for i, j in enumerate(k.by_sample == sampNum) if j]
    #     # labels_samp = np.asarray(pickle_load(f'dbLabels_samp{sampNum}_{name}_', dir_data))
        
    #     # sample indexs which are the full dataset - filter db labels by it
    #     # K = pickle_load(f'k{sampNum}_dict', dir_data)['k'].reset_index(drop=True)
    #     ind_samp = [i for i, j in zip(k.index,k['Clust'] == sampNum) if j]
    #     # corresponding indexes in the full data set - where each point in the full umap
    #     labels_samp = np.asarray(k['kmean_labels'])[ind_samp]
    #     # colors_samp = pickle_load(f'dbColors_samp{sampNum}_{name}_', dir_data)
    #     colors_samp = np.vstack(([0,0,0,1],colors))
    #     # pickle_dump(f'dbColors_{figname}', colors, dir_data)

    #     drawUMAPbySample(umapData, k, ind_samp, labels_samp, settings,colors_samp,
    #                         title=title+f' UMAP - by_sample={sampNum} clusters',
    #                         Figname='4_'+figname+f'UMAP_sample{sampNum}_by_sample_clusters')

## features on UMAP


shap

In [ ]:
if args['visualize'] and args['plotUMAP']:
    ind = labels !=-1 # drop unclustered rows (clust==-1)
    accuracy = xg_classification_shap(CAll[featureGroup].copy(),labels.copy(),ind,colors,
                                    settings,figname='5_'+figname+'shap')
    print(f"accuracy {j}_{group_ind}: {accuracy}%")

plot Clusters

In [ ]:
if args['visualize'] and args['plotUMAP']:
    plotClusters(k, umapData, labels, colors, NamesAll, settings,
                title='Clusters ' + title,
                figname='1_'+figname+'Clusters')

plot features on umap 

In [ ]:
if args['visualize'] and args['plotUMAP']:
    cols = NamesAll.copy()
    # cols = [NamesAll[0]]
    for i in  ['CyclinB1', 'pRB','H3S28p','DNA2']:
        if i not in cols and i in CAll:
            cols.append(i)
            # print(f'added {i} to NamesAll')

    limits = np.hstack((np.floor([2*x for x in [umapData[:,0].min(),umapData[:,1].min()]])/2,\
                        np.ceil([2*x for x in [umapData[:,0].max(),umapData[:,1].max()]])/2))
    ind = k.index
    drawUMAP(umapData[ind], cols, CAll.loc[ind], settings,limits = limits,
             title='UMAP ' + title,
             Figname='2_'+figname+'UMAP_')
    
    ind = list(ind[k.Clust != -1])#remove unclustered -1
    # ind=np.random.choice(ind, replace = False, size = int(len(ind)/4)) #dilute sub sample so view is sparse
    drawUMAP(umapData[ind], cols, CAll.loc[ind], settings,limits = limits,
            title='UMAP ' + title,
            Figname='2_'+figname+'UMAP_without_unclustered(-1)_')
    # ['CyclinB1', 'pRB','H3S28p','DNA2']


plot histograms

In [ ]:
from plot_functions import *    
if '.2a' in j and group_ind==2:  
    import seaborn as sns
    K = {
        'pRB+':k[k['cellcycle_labels']==1].copy(),
        'pRB-':k[k['cellcycle_labels']==0].copy(),
        }
    plot_hist(K,NamesAll.copy(),figures ={'pRB+':1,'pRB-':1},settings = settings,numSubplots = 1,
        func = sns.kdeplot,title = 'Cellcycle',Figname = '6_Hist_Cellcycle_' ) 
    K = None

## scatter figures


In [ ]:
# if args['visualize']:
#     try:
#         scatter(k, 'MBD', 'DNA2', title, figname, settings)
#     except:
#         print('wanted features not in sample')
# # k.columns

## clear data


In [ ]:
# dbscanData, umapData,k, CAll,labels = None,None,None,None,None
# imList2pdf(dir_plots,j,['CellIden','EpiCols','CellIden+EpiCols'])
